# Importing the DataSet

In [1]:
img_size = [227,227] #List which stores the resolution
main_train = '../input/mit-split-crack/50/Train' #Stores the path of the train directory
main_test = '../input/mit-split-crack/50/Test' #Stores the path of the test directory

In [2]:
from tensorflow.keras.layers import Input,Dense,Flatten,Dropout
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Model Building

In [3]:
vgg_model = VGG16(include_top=False,
    weights="imagenet",
    input_shape=img_size + [3])

# Loading the VGG16 pre trained model

# include_top - this specifies whether the final layer before the output layer has to be include 
# or not. If included then there will be 1000 number of classes at the output.

# Weights are trained using imagenet

2021-12-07 04:49:29.479987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 04:49:29.601052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 04:49:29.602253: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 04:49:29.605010: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

58900480/58889256 [==============================] - 0s 0us/step


In [4]:
# To fix the weights of the pre trained model
for lay in vgg_model.layers:
    lay.trainable = False
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
# Flatten() is used to convert the last layer to vector or as fully connected
x = Flatten(name="first_flatten")(vgg_model.output)
# Dense() is a fully connected layer and produces 128 outputs.
# relu layer is used to convert the negative nubers to 0 and retains +ve as such.
pred = Dense(256, activation="relu")(x)
#Dropout() layer is used to randomly remove 50% of the networks.
pred = Dropout(0.5)(pred)
# Dense() layer is added such that it outputs only two classess 
# Softmax activation layer produces probabilities for different classess.
pred = Dense(2,activation='softmax')(pred)
# Model() is used to group layers
model = Model(inputs=vgg_model.input,outputs=pred)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# Image Augmentation

In [6]:
#ImageDataGenerator is used to modify the train and test images w.r.t the properties mentioned
train_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=20,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=20,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   horizontal_flip = True)

# flow_from_directory() is used to convert all the images in the specific directory
training_set = train_datagen.flow_from_directory(directory = main_train,
                                                 target_size = (227,227),
                                                 batch_size = 500,
                                                 )

test_set = test_datagen.flow_from_directory(directory = main_test,
                                                 target_size = (227,227),
                                                 batch_size = 500,
                                                 )

Found 15904 images belonging to 2 classes.
Found 3958 images belonging to 2 classes.


# Model Fitting

In [7]:
# Loss function is used to find the errors or deviations in learning process
# Optimizer is used to optimize the input weights
# Metrics is used to measure the performance
model.compile(optimizer=Adam(learning_rate=0.10),
    loss="binary_crossentropy",
    metrics=['accuracy'])

In [8]:
# fit() is used to train the model 

mod = model.fit(
  training_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

2021-12-07 04:50:03.371665: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2


2021-12-07 04:50:15.293645: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2021-12-07 04:50:19.653008: W tensorflow/core/kernels/gpu_utils.cc:49] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.


32/32 [==============================] - 482s 14s/step - loss: 110.6304 - accuracy: 0.8121 - val_loss: 0.2991 - val_accuracy: 0.9454
Epoch 2/2
32/32 [==============================] - 336s 11s/step - loss: 0.5398 - accuracy: 0.9120 - val_loss: 0.1947 - val_accuracy: 0.9525


In [9]:
# To save the particular model in .h5 format
import tensorflow as tf
from tensorflow.keras.models import load_model
model.save('256_0.10_50%.h5')

import joblib
joblib.dump(mod.history,'256_0.10_50%')

['256_0.10_50%']